In [ ]:
from eventbrite import Eventbrite

In [ ]:
from settings import (EVENTBRITE_API_KEY, EVENTBRITE_CLIENT_SECRET, 
                      EVENTBRITE_PERSONAL_OAUTH_TOKEN, EVENTBRITE_ANON_OAUTH_TOKEN)

eventbrite = Eventbrite(EVENTBRITE_PERSONAL_OAUTH_TOKEN)
eventbrite_anon = Eventbrite(EVENTBRITE_ANON_OAUTH_TOKEN)

# user attached to personal token

In [ ]:
user = eventbrite.get_user()  # Not passing an argument returns yourself
user['id'], user['name']

No user attached to anonymous token

In [ ]:
eventbrite_anon.get_user()

In [ ]:
# directly 

In [ ]:
user = eventbrite.get('/users/me')
user

In [ ]:
owned_events = eventbrite.get('/users/me/owned_events')
owned_events

In [ ]:
owned_events['events'][0]

In [ ]:
from pandas import DataFrame, Series

In [ ]:
df = DataFrame(owned_events['events'])
df

In [ ]:
owned_events['events'][0]

In [ ]:
# attendees

test_event_id = owned_events['events'][0]['id']
attendees = eventbrite.get('/events/{}/attendees/'.format(test_event_id))


In [ ]:
attendee = attendees['attendees'][0]
profile = attendee['profile']
(profile['first_name'], profile['last_name'], profile['email'])

In [ ]:
owned_events['events'][0]

In [ ]:
new_event_data = {'capacity': 50,
 'capacity_is_custom': False,
 'category_id': None,
 'changed': '2017-12-07T18:23:46Z',
 'created': '2017-12-07T18:21:55Z',
 'currency': 'USD',
 'description': {'html': '<P>[This is only a test event.]</P>\n<P>A month after the BIDS Hack Day, we come back to take what was accomplished to the next level.</P>',
  'text': '[This is only a test event.]\nA month after the BIDS Hack Day, we come back to take what was accomplished to the next level.'},
 'end': {'local': '2018-01-08T17:00:00',
  'timezone': 'America/Los_Angeles',
  'utc': '2018-01-09T01:00:00Z'},
 'format_id': None,
 'hide_end_date': False,
 'hide_start_date': False,
 'id': '41116552716',
 'invite_only': True,
 'is_free': True,
 'is_locked': False,
 'is_reserved_seating': False,
 'is_series': False,
 'is_series_parent': False,
 'listed': False,
 'locale': 'en_US',
 'logo': {'aspect_ratio': '2',
  'crop_mask': {'height': 512, 'top_left': {'x': 0, 'y': 128}, 'width': 1024},
  'edge_color': '#89abc7',
  'edge_color_set': True,
  'id': '38502347',
  'original': {'height': 768,
   'url': 'https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F38502347%2F719611263%2F1%2Foriginal.jpg?s=d44f0cbba38ead505ab21cef56469c1d',
   'width': 1024},
  'url': 'https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F38502347%2F719611263%2F1%2Foriginal.jpg?h=200&w=450&rect=0%2C128%2C1024%2C512&s=efe477b7e85acacbde2e0c09eedf51ea'},
 'logo_id': '38502347',
 'name': {'html': '[TEST] Hacking event management systems II',
  'text': '[TEST] Hacking event management systems II'},
 'online_event': False,
 'organizer_id': '15949757195',
 'privacy_setting': 'unlocked',
 'resource_uri': 'https://www.eventbriteapi.com/v3/events/41116552716/',
 'shareable': False,
 'show_remaining': True,
 'source': 'create_2.0',
 'start': {'local': '2018-01-08T09:00:00',
  'timezone': 'America/Los_Angeles',
  'utc': '2018-01-08T17:00:00Z'},
 'status': 'live',
 'subcategory_id': None,
 'tx_time_limit': 480,
 'url': 'https://www.eventbrite.com/e/test-hacking-event-management-systems-ii-tickets-41116552716',
 'venue_id': '22421902',
 'version': '3.0.0'}

In [ ]:
# https://www.eventbrite.com/developer/v3/endpoints/events/#ebapi-post-events

new_event_data = {

  'event': {
    
          'name': {
               'html': '[TEST] Hacking event management systems III'
          },

          'start': {
            'utc': '2018-02-08T17:00:00Z',
            'timezone': 'America/Los_Angeles',  
          },

         'end': {
          'utc': '2018-02-09T01:00:00Z',
          'timezone': 'America/Los_Angeles',
          },

          'currency': 'USD',

          'invite_only': True

        }
    
    }

In [ ]:
new_event = eventbrite.post_event(new_event_data)
new_event

# pagination

[Pagination support by njalerikson · Pull Request #41 · eventbrite/eventbrite-sdk-python](https://github.com/eventbrite/eventbrite-sdk-python/pull/41)

In [ ]:
# https://www.eventbrite.com/developer/v3/endpoints/events/#ebapi-get-events-search

events = eventbrite.get('/events/search', data = {
    'sort_by':'date',
    'location.address':'Berkeley, CA',
    'price':'free'
})


In [ ]:
events.pagination

[Paginated Responses - Eventbrite Developer Center](https://www.eventbrite.com/developer/v3/api_overview/pagination/)

> All API endpoints which return multiple objects will return paginated responses; as well as the list of objects (which will usually be under a key like events or attendees, depending on the endpoint) there will also be a pagination key

In [ ]:
len(events['events'])

In [ ]:
def all_search(eventbrite, data):
    page_number = 1
    more_pages = True
 

    while more_pages:
        data['page'] = page_number
        events = eventbrite.get('/events/search', data=data)
        
        #print (events['error_description'])
        p = events.pagination
        print(p, events.keys())
        

        for event in events['events']:
            yield event
            
        if not p['has_more_items']:
            more_pages = False
        else:
            page_number += 1
        
    
    

In [ ]:
events = all_search(eventbrite, data = {
    'sort_by':'date',
    'location.address':'Berkeley, CA',
    'price':'free'
})

In [ ]:
e = list(events)

In [ ]:
len(e)